Stock Number

In [ ]:
# stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
#             "2105", "2207", "2227", "2301", "2303", "2308", "2317", "2327",
#             "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
#             "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
#             "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
#             "4904", "4938", "5871", "5876", "5880", "6505", "9904", "9910"]
stockNum = ["1101", "1102", "1216"]

Package Installation

In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
%mkdir 0050_inner_stock

GPU Environment Setting

In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import *
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    # tf.config.experimental.set_virtual_device_configuration(gpu0,
    # [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

Import Package

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import talib
from talib import *
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import mpl_finance as mpf
import seaborn as sns
import datetime as datetime
import talib
from sklearn import preprocessing
import csv
import requests
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
import numpy as np
#from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import pandas as pd
import statistics
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
import tensorflow as tf
from tensorflow.python.keras.layers import  Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

ModuleNotFoundError: ignored

Preparation for data collection

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
start = '2005/01/01'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

Save data into files

In [ ]:
for stockno in stockNum:
    # load 進 五大價格 一個量
    df = pdr.DataReader(str(stockno)+'.TW', 'yahoo', start=start, end=end)
    df.index = df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
    stock_time = str(df.index[0]).split(" ")[0]
    #load in 融資融券
    url = "https://api.finmindtrade.com/api/v3/data"
    parameter = {
        "dataset": "TaiwanStockMarginPurchaseShortSale",
        "stock_id": str(stockno),
        "date": stock_time,
    }
    data = requests.get(url, params=parameter)
    data = data.json()
    data = pd.DataFrame(data['data'])
    #if 沒資料 則跳過這張股票
    if data.empty == True:
        print(str(stockno)+" can't find data !")
        continue
    ratio = []
    b_total = []
    s_total = []
    for i in range(len(data)):
        b_total.append(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i])
        s_total.append(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])
        ratio.append(float(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])/(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i]+1))
    data['b_total'] = b_total
    data['s_total'] = s_total
    data['b_s_ratio'] = ratio
    data = data.set_index('date')
    #合併這兩個表格
    new_df = df.join(data)
    new_df = new_df.drop(['stock_id'], axis=1)
    new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
    #處理三大法人投資情況 視情況加入
    url = "https://api.finmindtrade.com/api/v3/data"
    parameter = {
        "dataset": "InstitutionalInvestorsBuySell",
        "stock_id": str(stockno),
        "date": stock_time,
    }
    data = requests.get(url, params=parameter)
    data = data.json()
    data = pd.DataFrame(data['data'])
    if data.empty == True:
        print(str(stockno)+" can't find data !")
        continue
    df = pd.DataFrame()
    z = 0 
    i = 0
    while i < len(data)  :
        a = data['date'][i]
        count = 0
        sell  = 0
        buy = 0 
        if i+6 < len(data)-1:
            for j  in range(i, i+6):
                if data['date'][j]== a :
                    count += 1
                    sell += data['sell'][j]
                    buy += data['buy'][j]
        else:
            for j in range(i, len(data)):
                if data['date'][j] == a :
                    count += 1
                    sell += data['sell'][j]
                    buy += data['buy'][j]
        df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
        i+=count
    # combine
    new_df = new_df.join(df.set_index('date'))
    new_df = new_df.dropna(axis=0,how='any')
    new_df.reset_index(drop=True, inplace=True)
    days = 1
    ans = [0]*days
    for i in range(days,len(new_df)):
        Pi = new_df['Close'][i]
        Pj = new_df['Close'][i-days]
        if Pi > Pj*1.03: y = 1
        elif Pi < Pj*0.97: y = -1
        else: y = 0
        ans.append(y)
    new_df['y_label']= ans
    new_df = new_df.drop(['Note'], axis=1)
    #store file
    new_df.to_csv('./0050_inner_stock/'+str(stockno)+'.csv')
    #new_df.to_csv(str(stockno)+'.csv')
    print(str(stockno)+" complete....")

1101 complete....
1102 complete....
1216 complete....


Exponential Smoothing & One-Hot Encoding

In [ ]:
def exponential_smoothing(alpha, numdata):
  new_numdata = []
  for i in range(0, len(numdata)):
    if i == 0 : 
      new_numdata.append(numdata[i])
      tmp = new_numdata[i]
      continue;
    new_numdata.append( alpha*tmp + (1-alpha)*numdata[i] )
    tmp = new_numdata[i]
  return new_numdata

classes =['-1', '0', '1']
def to_onelist(text):
    onehot = [0] * 3
    onehot[ int( classes.index(str(text)) ) ] = 1
    label_list = np.array(onehot)
    return label_list

In [1]:
stockNum = ["1101", "1102", "1216"]
# , "1301", "1326", "1402", "2002",
#             "2105", "2207", "2227", "2301", "2303", "2308", "2317", "2327",
#             "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
#             "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
#             "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
#             "4904", "4938", "5871", "5876", "5880", "6505", "9904", "9910"]
#對資料做標準化
def normal_train(df):
    for col in df.columns:
        if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
        else:
            mean = statistics.mean(df[str(col)])
            stdev = statistics.stdev(df[str(col)])
            if stdev == 0: stdev = 1
            def test(s): 
                return (float(s)-mean)/stdev
            df[str(col)] = df[str(col)].map(test)
    return df

all_data = []
y = [] 
for stockno in stockNum:
    if int(stockno) == 2227: continue
    df = pd.read_csv('./0050_inner_stock/'+str(stockno)+'.csv')
    df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
    df = df.drop(['Date'], axis=1)
    df = df.drop(['MarginPurchaseLimit'], axis=1)
    df = df.drop(['ShortSaleLimit'], axis=1)
    df = df.dropna(axis=0, how='any')
    df.reset_index(drop=True, inplace=True)
    #exponential smoothing
    for columnName in df.columns:
      # print('Colunm Name : ', columnName)
      # print('Column Contents : \n', new_df[str(columnName)])
      if str(columnName) == 'y_label' : continue
      df[str(columnName)] = exponential_smoothing(0.2, list(df[str(columnName)]))
      # print(new_df[str(columnName)])
      # print('===')
    days_before = 30
    for i in range(days_before, len(df)):
        a = normal_train(df[i-days_before : i])
        all_data.append( np.array(np.array(a).tolist()[0: days_before]).reshape(days_before,len(a.columns)))
    y = y + list(df['y_label'][days_before:]) 
    print(str(stockno)+" loaded....")

NameError: ignored

In [ ]:
y = list(y)
y_one_hot = []
for i in range(len(y)):
    y_one_hot.append(to_onelist(str(y[i])))

X_train, X_val, y_train, y_val = train_test_split( all_data, y_one_hot, test_size=0.4, random_state=69)
X_val, X_val, y_test, y_test = train_test_split( X_val, y_val, test_size=0.5, random_state=69)

In [ ]:
def count_label_dist(arr):
  count_1 = 0 
  count_0 = 0 
  count_n1 = 0
  for i in range(len(arr)):
    if list(arr[i])== list([0,0,1]): count_1 += 1
    elif list(arr[i])== list([0,1,0]): count_0 += 1
    else: count_n1 += 1
  print(count_n1, count_0, count_1)

count_label_dist(y_train)
count_label_dist(y_val)
count_label_dist(y_test)

75 3480 81
45 2320 59
37 2093 52


In [ ]:
len_tr = len(X_train) 
len_val = len(X_val)
print(len_tr,len_val)
print(np.array(X_train).shape , np.array(y_train).shape)
X_train = np.array(X_train)[:,np.newaxis]
X_val = np.array(X_val)[:,np.newaxis]
y_train = np.array(y_train)[:,np.newaxis]
y_val = np.array(y_val)[:,np.newaxis]
X_train = X_train.reshape( len_tr, 30, 23)
X_val = X_val.reshape( len_val, 30, 23)
print(np.array(X_train).shape , np.array(y_train).shape)

3636 2424
(3636, 30, 23) (3636, 3)
(3636, 30, 23) (3636, 1, 3)


In [ ]:
# def mycrossentropy(y_true, y_pred):
#     #loss1 = K.categorical_crossentropy(y_true, y_pred)
#     for (y_t, y_p) in (y_true, y_pred):
#       if y_t != y_p && y_t != [0,1,0]:
#         loss += K.categorical_crossentropy(y_t, y_p)*30
#       else: 
#         loss += K.categorical_crossentropy(y_t, y_p)
#     return loss #(1-e)*loss1 + e*loss2

def mycrossentropy(y_true, y_pred):
    # y_true = K.print_tensor(y_true, message='y_true = ')
    # y_pred = K.print_tensor(y_pred, message='y_pred = ')
    loss1 = K.categorical_crossentropy(y_true, y_pred)
    loss = 0
    for (y_t, y_p) in zip(*y_true, *y_pred):
        loss += K.categorical_crossentropy(y_t, y_p)
    return loss1

    #print(K.get_value(y_true))
    # for i,j in zip( K.get_value(y_true), K.get_value(y_pred)):
    #    d = tf.Print(d, [y_true], "Inside loss function")
    # for (y_t, y_p) in zip(y_true, y_pred):
    #   if y_t != y_p && y_t != [0,1,0]:
    #     loss += K.categorical_crossentropy(y_t, y_p)*30
    #   else: 
    #     loss += K.categorical_crossentropy(y_t, y_p)
    return loss1 #(1-e)*loss1 + e*loss2

In [ ]:
def new_multi1_model(input_shape, n_classes):
    model = Sequential()
    model.add(Dense(23,input_shape=(input_shape[0],input_shape[1])))
    
    model.add(BatchNormalization())
    model.add(Conv1D(filters=80,kernel_size=3,padding='valid',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    #model.add(MaxPooling1D(pool_size=2,padding='valid'))
    #model.add(Dropout(0.2))
    
    model.add(BatchNormalization())
    model.add(Conv1D(filters=80,kernel_size=5,padding='valid',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    #model.add(MaxPooling1D(pool_size=2,padding='valid'))
    #model.add(Dropout(0.2))
    
    model.add(BatchNormalization())
    model.add(Conv1D(filters=80,kernel_size=7,padding='valid',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    #model.add(MaxPooling1D(pool_size=2,padding='valid'))
    #model.add(Dropout(0.2))
    
    model.add(BatchNormalization())
    model.add(Flatten())
    #model.add(Dropout(0.2))
    model.add(Dense(10,activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    
    model.add(BatchNormalization())
    model.add(Dense(n_classes, activation='softmax', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.compile(loss="categorical_crossentropy", optimizer=Adamax(learning_rate=0.0001), metrics=['accuracy'])
    #model.compile(loss=mycrossentropy, optimizer=Adamax(learning_rate=0.0001), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
tf.keras.backend.clear_session()
model = new_multi1_model((30, 23), 3)
#model.run_eagerly = True
callback = EarlyStopping(monitor="val_loss", patience=25, verbose=1, mode="auto")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30, 23)            552       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 23)            92        
_________________________________________________________________
conv1d (Conv1D)              (None, 28, 80)            5600      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 80)            320       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24, 80)            32080     
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 80)            320       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 18, 80)            4

In [ ]:
# #將train 做 shuffle
# from random import shuffle
# N = len(X_train)
# print("shuffle長度： ", N)
# ind_list = [i for i in range(N)]
# shuffle(ind_list)
# #X_train_shuffle = X_train[ind_list, :,:]
# #y_train_shuffle = y_train[ind_list,]

X_train_shuffle = X_train
y_train_shuffle = y_train

y_train_shuffle = y_train_shuffle.reshape(len_tr,3)
y_val = y_val.reshape(len_val,3)
print(X_train_shuffle.shape)
print(y_train_shuffle.shape)
print(X_val.shape)
print(y_val.shape)
class_weight = { 0: 30, 1: 1., 2: 30}
history = model.fit(X_train_shuffle, y_train_shuffle , epochs=150, batch_size=256, validation_data=(X_val, y_val), class_weight=class_weight, callbacks=[callback])
#history = model.fit(X_train_shuffle, y_train_shuffle , epochs=150, batch_size=256, validation_data=(X_val, y_val), callbacks=[callback])

(3636, 30, 23)
(3636, 3)
(2424, 30, 23)
(2424, 3)
Epoch 1/150
15/15 [==============================] - 0s 14ms/step - loss: 2.4009 - accuracy: 0.9480 - val_loss: 0.4915 - val_accuracy: 0.9402
Epoch 2/150
15/15 [==============================] - 0s 13ms/step - loss: 1.9986 - accuracy: 0.9521 - val_loss: 0.5176 - val_accuracy: 0.9307
Epoch 3/150
15/15 [==============================] - 0s 13ms/step - loss: 1.7291 - accuracy: 0.9524 - val_loss: 0.5301 - val_accuracy: 0.9249
Epoch 4/150
15/15 [==============================] - 0s 13ms/step - loss: 1.5211 - accuracy: 0.9521 - val_loss: 0.5287 - val_accuracy: 0.9237
Epoch 5/150
15/15 [==============================] - 0s 12ms/step - loss: 1.3401 - accuracy: 0.9508 - val_loss: 0.5172 - val_accuracy: 0.9249
Epoch 6/150
15/15 [==============================] - 0s 12ms/step - loss: 1.2391 - accuracy: 0.9505 - val_loss: 0.5092 - val_accuracy: 0.9262
Epoch 7/150
15/15 [==============================] - 0s 12ms/step - loss: 1.1094 - accuracy: 0.953